<a href="https://colab.research.google.com/github/mearsdj/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-28 21:22:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.15MB/s    in 0.2s    

2022-08-28 21:22:54 (6.15 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|                Y|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|                Y|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|                Y|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|                Y|
| RNYLPX611NB7Q|          5|            0|          0|   N|                Y|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|                Y|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|                Y|
|R18WWEK8OIXE30|          5|            0|          0|   N|                Y|
|R1EGUNQON2J277|          1|            0|          0|   N|                Y|
|R21Z4M4L98CPU2|          5|            0|          0|   N|     

In [7]:

vine_df_over20votes = vine_df.filter(vine_df.total_votes>20)
vine_df_over20votes.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|                N|
|R32PHGNOUNJVY2|          1|          111|        150|   N|     

In [8]:
#filter for helpful votes >50% of total
vine_df_over20votes_helpful = vine_df_over20votes.filter((vine_df_over20votes.helpful_votes/vine_df_over20votes.total_votes) > 0.5)
vine_df_over20votes_helpful.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|                N|
|R32PHGNOUNJVY2|          1|          111|        150|   N|     

In [9]:
filtered_df_in_vine = vine_df_over20votes_helpful.filter(vine_df_over20votes_helpful.vine=='Y')
filtered_df_not_in_vine = vine_df_over20votes_helpful.filter(vine_df_over20votes_helpful.vine=='N')

In [29]:
total_reviews_in_vine = filtered_df_in_vine.agg({'review_id':'count'}).withColumnRenamed('count(review_id)','Total Reviews in Vine')
fivestar_reviews_in_vine = filtered_df_in_vine.filter(filtered_df_in_vine.star_rating==5).agg({'review_id':'count'}).withColumnRenamed('count(review_id)','Total Five Star Reviews in Vine')
total_reviews_in_vine.show()
fivestar_reviews_in_vine.show()
review_df = total_reviews_in_vine.join(fivestar_reviews_in_vine)
review_df.withColumn('% of Five Star Ratings - In Vine',review_df['Total Five Star Reviews in Vine']/review_df['Total Reviews in Vine']).show()
#No vine reviews for gift cards!! :)
total_reviews_not_in_vine = filtered_df_not_in_vine.agg({'review_id':'count'}).withColumnRenamed('count(review_id)','Total Reviews not in Vine')
fivestar_reviews_not_in_vine = filtered_df_not_in_vine.filter(filtered_df_not_in_vine.star_rating==5).agg({'review_id':'count'}).withColumnRenamed('count(review_id)','Total Five Star Reviews not in Vine')
total_reviews_not_in_vine.show()
fivestar_reviews_not_in_vine.show()
review_df2 = total_reviews_not_in_vine.join(fivestar_reviews_not_in_vine)
review_df2.withColumn('% of Five Star Ratings - not In Vine',review_df2['Total Five Star Reviews not in Vine']/review_df2['Total Reviews not in Vine']).show()

+---------------------+
|Total Reviews in Vine|
+---------------------+
|                    0|
+---------------------+

+-------------------------------+
|Total Five Star Reviews in Vine|
+-------------------------------+
|                              0|
+-------------------------------+

+---------------------+-------------------------------+--------------------------------+
|Total Reviews in Vine|Total Five Star Reviews in Vine|% of Five Star Ratings - In Vine|
+---------------------+-------------------------------+--------------------------------+
|                    0|                              0|                            null|
+---------------------+-------------------------------+--------------------------------+

+-------------------------+
|Total Reviews not in Vine|
+-------------------------+
|                      340|
+-------------------------+

+-----------------------------------+
|Total Five Star Reviews not in Vine|
+-----------------------------------+
|      

In [30]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
# df.show()

# Create the vine_table. DataFrame
vine_df_outdoor = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])

vine_df_outdoor_filtered = vine_df_outdoor.filter(vine_df_outdoor.total_votes>20)

#filter for helpful votes >50% of total
vine_df_outdoor_filtered = vine_df_outdoor_filtered.filter((vine_df_outdoor_filtered.helpful_votes/vine_df_outdoor_filtered.total_votes) > 0.5)

#calc percent in vine
filtered_df_in_vine = vine_df_outdoor_filtered.filter(vine_df_outdoor_filtered.vine=='Y')
filtered_df_not_in_vine = vine_df_outdoor_filtered.filter(vine_df_outdoor_filtered.vine=='N')

total_reviews_in_vine = filtered_df_in_vine.agg({'review_id':'count'})
fivestar_reviews_in_vine = filtered_df_in_vine.filter(filtered_df_in_vine.star_rating==5).agg({'review_id':'count'})
# pct_fivestar_reviews_in_vine = fivestar_reviews_in_vine/total_reviews_in_vine
total_reviews_not_in_vine = filtered_df_not_in_vine.agg({'review_id':'count'})
fivestar_reviews_not_in_vine = filtered_df_not_in_vine.filter(filtered_df_not_in_vine.star_rating==5).agg({'review_id':'count'})

In [31]:
total_reviews_in_vine.show()
fivestar_reviews_in_vine.show()

total_reviews_not_in_vine.show()
fivestar_reviews_not_in_vine.show()

+----------------+
|count(review_id)|
+----------------+
|             103|
+----------------+

+----------------+
|count(review_id)|
+----------------+
|              55|
+----------------+

+----------------+
|count(review_id)|
+----------------+
|           37712|
+----------------+

+----------------+
|count(review_id)|
+----------------+
|           19810|
+----------------+



In [34]:
total_reviews_not_in_vine=total_reviews_not_in_vine.withColumnRenamed('count(review_id)','Total Non-vine Reviews')
fivestar_reviews_not_in_vine=fivestar_reviews_not_in_vine.withColumnRenamed('count(review_id)','Total Non-vine Five Star Reviews')
total_reviews_in_vine=total_reviews_in_vine.withColumnRenamed('count(review_id)','Total In-vine Reviews')
fivestar_reviews_in_vine=fivestar_reviews_in_vine.withColumnRenamed('count(review_id)','Total In-vine Five Star Reviews')
review_df3=total_reviews_not_in_vine.join(fivestar_reviews_not_in_vine)
review_df3=review_df3.withColumn('% of 5 star reviews not in vine',review_df3['Total Non-vine Five Star Reviews']/review_df3['Total Non-vine Reviews'])
review_df3=review_df3.join(total_reviews_in_vine)
review_df3=review_df3.join(fivestar_reviews_in_vine)
review_df3=review_df3.withColumn('% of 5 star reviews in vine',review_df3['Total In-vine Five Star Reviews']/review_df3['Total In-vine Reviews'])



In [35]:
review_df3.show()


+----------------------+--------------------------------+-------------------------------+---------------------+-------------------------------+---------------------------+
|Total Non-vine Reviews|Total Non-vine Five Star Reviews|% of 5 star reviews not in vine|Total In-vine Reviews|Total In-vine Five Star Reviews|% of 5 star reviews in vine|
+----------------------+--------------------------------+-------------------------------+---------------------+-------------------------------+---------------------------+
|                 37712|                           19810|              0.525296987696224|                  103|                             55|         0.5339805825242718|
+----------------------+--------------------------------+-------------------------------+---------------------+-------------------------------+---------------------------+

